In [ ]:
!pip install gensim

In [ ]:
!pip install annoy

In [ ]:
!pip install pymorphy3

In [ ]:
!pip install stop-words

In [ ]:
import os
import string
import annoy
import codecs

from pymorphy3 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec

import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
question = None # Здесь будем хранить текущий вопрос
written = False # Флаг: записали ли мы уже ответ на текущий вопрос
# Пути к файлам
otvety_path = "/content/drive/MyDrive/Colab Notebooks/Otvety.txt"
prepared_path = "/content/drive/MyDrive/Colab Notebooks/prepared_answers.txt"
#Мы идем по всем записям, берем первую строку как вопрос
# и после знака --- находим ответ
with codecs.open(prepared_path,"w", "utf-8") as fout:
    with codecs.open(otvety_path, "r", "utf-8") as fin:
        for line in tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

print("Файл prepared_answers.txt создан!")


0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Загружаем датасет с товарами
products_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ProductsDataset.csv')

print("ИНФОРМАЦИЯ О ДАТАСЕТЕ ТОВАРОВ:")
print(f"Размер: {products_df.shape[0]} строк, {products_df.shape[1]} колонок")
print(f"\n Колонки: {products_df.columns.tolist()}")
print(f"\n Первые 3 товара:")
print(products_df.head(3))

# Проверяем, есть ли пропущенные значения
print(f"\n Пропущенные значения:")
print(products_df.isnull().sum())

In [ ]:
# Очистка данных товарах

# 1. Удаляем строки без product_id или title (они нам не нужны)
print(f"До очистки: {len(products_df)} товаров")
products_df = products_df.dropna(subset=['product_id', 'title'])
print(f"После очистки: {len(products_df)} товаров")

# 2. Переименовываем колонку с опечаткой
products_df = products_df.rename(columns={'descrirption': 'description'})

# 3. Заполняем пустые описания названием (чтобы не было NaN)
products_df['description'] = products_df['description'].fillna(products_df['title'])

# 4. Оставляем только нужные колонки
products_df = products_df[['product_id', 'title', 'description']]

print(f"\nДанные очищены!")
print(f"Итого товаров для обучения: {len(products_df)}")
print(f"\n Проверка: есть ли 'Юбка детская ORBY'?")
test_row = products_df[products_df['title'] == 'Юбка детская ORBY']
if not test_row.empty:
    print(f"✓ Найдена! ID: {test_row.iloc[0]['product_id']}")
else:
    print("Не найдена")

# 5. Показываем примеры
print(f"\n Первые 3 товара после обработки:")
print(products_df.head(3))

In [ ]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

def preprocess_for_tfidf(line):
    """
    Препроцессинг для TF-IDF классификатора
    Возвращает строку вместо списка
    """
    return " ".join(preprocess_txt(line))


In [ ]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with codecs.open(otvety_path, "r", "utf-8") as fin:
    for line in tqdm(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break
print(f" Собрано {len(sentences)} предложений из вопросов")

In [ ]:

# Добавляем товары в обучающую выборку для Word2Vec

products_added = 0

for _, row in tqdm(products_df.iterrows(), total=len(products_df), desc="Обработка товаров"):
    # Добавляем название товара
    title_words = preprocess_txt(row['title'])
    if len(title_words) > 0:
        sentences.append(title_words)
        products_added += 1

    # Добавляем описание товара
    desc_words = preprocess_txt(row['description'])
    if len(desc_words) > 0:
        sentences.append(desc_words)
        products_added += 1

print(f"Добавлено {products_added} предложений из товаров")
print(f"Всего предложений для Word2Vec: {len(sentences)}")

# ФИЛЬТРАЦИЯ И ОБУЧЕНИЕ WORD2VEC
print("\n🧹 Фильтруем короткие предложения...")
sentences = [i for i in sentences if len(i) > 2]
print(f"После фильтрации: {len(sentences)} предложений")

print("\n Обучаем Word2Vec модель...")
model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5, workers=4)
model.save("/content/w2v_model")
print("Word2Vec модель обучена и сохранена!")

In [ ]:
# === СОЗДАНИЕ ДАТАСЕТА ДЛЯ КЛАССИФИКАТОРА ===
from sklearn.model_selection import train_test_split

texts = []   # Тексты для обучения
labels = []  # Метки: 1 = товар, 0 = болталка

# === КЛАСС 1: ТОВАРЫ ===
for _, row in tqdm(products_df.iterrows(), total=len(products_df), desc="Товары"):
    # Добавляем название + описание (полный текст)
    full_text = f"{row['title']} {row['description']}"
    texts.append(preprocess_for_tfidf(full_text))
    labels.append(1)

    # Добавляем только название (короткие запросы)
    texts.append(preprocess_for_tfidf(row['title']))
    labels.append(1)

print(f"Добавлено {len([l for l in labels if l == 1])} примеров товаров")

# === КЛАСС 0: ВОПРОСЫ (БОЛТАЛКА) ===
with codecs.open(prepared_path, "r", "utf-8") as f:
    for line in tqdm(f, desc="Вопросы"):
        parts = line.split("\t")
        if len(parts) >= 2:
            question = parts[0]
            texts.append(preprocess_for_tfidf(question))
            labels.append(0)

print(f"Добавлено {len([l for l in labels if l == 0])} примеров вопросов")

# === РАЗДЕЛЕНИЕ НА TRAIN/VAL ===
print(f"\n Всего примеров: {len(texts)}")
print(f"   - Товары: {sum(labels)}")
print(f"   - Вопросы: {len(labels) - sum(labels)}")

X_train, X_val, y_train, y_val = train_test_split(
    texts, labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

print(f"\n Данные разделены:")
print(f"   - Train: {len(X_train)} примеров")
print(f"   - Val: {len(X_val)} примеров")

In [ ]:
# === ОБУЧЕНИЕ КЛАССИФИКАТОРА ===
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
import pickle

# === ШАГ 1: ВЕКТОРИЗАЦИЯ ТЕКСТА (TF-IDF) ===
vectorizer = TfidfVectorizer(
    max_features=5000,      # Берём топ-5000 самых важных слов
    ngram_range=(1, 2),     # Используем uni-граммы и bi-граммы
    min_df=2                # Слово должно встречаться минимум в 2 документах
)

X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)
print(f"Размер обучающей матрицы: {X_train_vec.shape}")

# === ШАГ 2: ОБУЧЕНИЕ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ ===
classifier = LogisticRegression(
    max_iter=1000,
    random_state=42,
    class_weight='balanced',  # Балансируем классы (товаров меньше)
    n_jobs=-1                 # Используем все ядра процессора
)
classifier.fit(X_train_vec, y_train)

# === ШАГ 3: ОЦЕНКА КАЧЕСТВА НА ВАЛИДАЦИИ ===
print("\n РЕЗУЛЬТАТЫ НА ВАЛИДАЦИИ:\n")
y_pred = classifier.predict(X_val_vec)

print(f"Accuracy: {accuracy_score(y_val, y_pred):.4f}\n")
print("Classification Report:")
print(classification_report(y_val, y_pred, target_names=['Болталка (0)', 'Товар (1)']))

# === ШАГ 4: СОХРАНЕНИЕ МОДЕЛЕЙ ===
with open('/content/classifier.pkl', 'wb') as f:
    pickle.dump(classifier, f)
with open('/content/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
print(" Классификатор и векторизатор сохранены!")

In [ ]:
# === СОЗДАНИЕ ИНДЕКСА ДЛЯ ТОВАРОВ ===

def text_to_vector(text, model):
    """
    Преобразует текст в вектор Word2Vec:
    1. Препроцессим текст
    2. Берём векторы всех слов
    3. Усредняем их
    """
    words = preprocess_txt(text)
    vectors = []

    for word in words:
        if word in model.wv:
            vectors.append(model.wv[word])

    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(100)

# Создаём Annoy индекс
product_index = annoy.AnnoyIndex(100, 'angular')
product_map = {}  # Словарь: индекс -> (product_id, title)

# Добавляем товары в индекс
for idx, row in tqdm(products_df.iterrows(), total=len(products_df), desc="Индексация"):
    # Векторизуем название товара
    vector = text_to_vector(row['title'], model)
    product_index.add_item(idx, vector)
    product_map[idx] = (row['product_id'], row['title'])

# Строим индекс
product_index.build(10)
product_index.save('/content/products.ann')

# Сохраняем маппинг
with open('/content/product_map.pkl', 'wb') as f:
    pickle.dump(product_map, f)

print(f"Индекс товаров создан: {len(product_map)} товаров")

# === ТЕСТ ИНДЕКСА ===
test_query = "Юбка детская"
test_vector = text_to_vector(test_query, model)
nearest = product_index.get_nns_by_vector(test_vector, 3)

print(f"Запрос: '{test_query}'")
print("Найденные товары:")
for i, idx in enumerate(nearest, 1):
    pid, title = product_map[idx]
    print(f"  {i}. {title} (ID: {pid})")

In [ ]:
# === СОЗДАНИЕ ИНДЕКСА ДЛЯ БОЛТАЛКИ ===

chat_index = annoy.AnnoyIndex(100, 'angular')
chat_map = {}  # Словарь: индекс -> ответ
counter = 0

#Добавляем вопросы в индекс
with codecs.open(prepared_path, "r", "utf-8") as f:
    for line in tqdm(f, desc="Индексация вопросов"):
        parts = line.split("\t")
        if len(parts) >= 2:
            question = parts[0]
            answer = parts[1]

            # Векторизуем вопрос
            question_words = preprocess_txt(question)
            vector = np.zeros(100)
            n_w2v = 0

            for word in question_words:
                if word in model.wv:
                    vector += model.wv[word]
                    n_w2v += 1

            if n_w2v > 0:
                vector = vector / n_w2v

            chat_index.add_item(counter, vector)
            chat_map[counter] = answer
            counter += 1

# Строим индекс
print("\n🏗️ Строим индекс...")
chat_index.build(10)
chat_index.save('/content/speaker.ann')

# Сохраняем маппинг
with open('/content/chat_map.pkl', 'wb') as f:
    pickle.dump(chat_map, f)

print(f"Индекс болталки создан: {counter} вопросов")

# === ТЕСТ ИНДЕКСА ===
print("\n Тест индекса болталки:")
test_question = "Где ключи"
test_vector = text_to_vector(test_question, model)
nearest_idx = chat_index.get_nns_by_vector(test_vector, 1)[0]
print(f"Запрос: '{test_question}'")
print(f"Ответ: {chat_map[nearest_idx][:100]}...")

In [ ]:
# === ГЛАВНАЯ ФУНКЦИЯ GET_ANSWER() ===

def get_answer(query):
    """
    Главная функция для получения ответа на запрос пользователя

    Алгоритм:
    1. Препроцессим запрос
    2. Классифицируем: товарный или нет?
    3. Если товарный → ищем похожий товар → возвращаем "product_id title"
    4. Если нет → ищем похожий вопрос → возвращаем ответ

    Args:
        query (str): Запрос пользователя

    Returns:
        str: Ответ (для товара: "product_id title", для болталки: текст ответа)
    """
    # ШАГ 1: Препроцессинг запроса
    preprocessed = preprocess_for_tfidf(query)

    # ШАГ 2: Классификация (товар или болталка?)
    query_vec = vectorizer.transform([preprocessed])
    is_product = classifier.predict(query_vec)[0]

    if is_product == 1:
        # ========== ТОВАРНЫЙ ЗАПРОС ==========
        # Преобразуем в вектор Word2Vec
        vector = text_to_vector(query, model)

        # Ищем ближайший товар в индексе
        nearest_idx = product_index.get_nns_by_vector(vector, 1)[0]

        # Получаем product_id и название
        product_id, title = product_map[nearest_idx]

        # Возвращаем в формате "product_id title"
        return f"{product_id} {title}"

    else:
        # ========== НЕПРОДУКТОВЫЙ ЗАПРОС (БОЛТАЛКА) ==========
        # Преобразуем в вектор Word2Vec
        vector = text_to_vector(query, model)

        # Ищем ближайший вопрос в индексе
        nearest_idx = chat_index.get_nns_by_vector(vector, 1)[0]

        # Возвращаем ответ на похожий вопрос
        answer = chat_map[nearest_idx].strip()
        return answer


# === ТЕСТИРОВАНИЕ ===
print(" ТЕСТИРОВАНИЕ ФУНКЦИИ get_answer()\n")
print("="*70)

test_queries = [
    "Юбка детская ORBY",
    "Ботильоны женские",
    "Как приготовить борщ",
    "Платье новое 46 размер"
]

for query in test_queries:
    answer = get_answer(query)
    print(f"\n Запрос: {query}")
    print(f" Ответ: {answer[:150]}...")
    print("-"*70)